https://www.ibm.com/developerworks/jp/linux/library/l-simpy/
https://pypi.python.org/pypi/simpy

In [6]:
import simpyx  # https://github.com/chaosmail/simpy-cython
import simpy
from progressbar import ProgressBar
from  datetime import datetime as dt

In [7]:
def clock(env, name, tick):
    while True:
        yield env.timeout(tick)

def run(env, sim_time):
    for name in ["clock%s"%i for i in range(10)]:
        env.process(clock(env, name, 0.5))
    stime = dt.now()
    env.run(until=sim_time)
    etime = dt.now()
    delta = etime - stime
    return delta.total_seconds()

In [8]:
SIM_SIZE=24*10000

In [9]:
env = simpy.Environment()
print("simpy: %f sec." % run(env, SIM_SIZE))

simpy: 16.135659 sec.


In [10]:
env = simpyx.Environment()
print("simpyx: %f sec." % run(env, SIM_SIZE))

simpyx: 7.241605 sec.


圧倒的にはやい。

### ついでにcython で関数作り直してみる

In [11]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [12]:
%%cython
from  datetime import datetime as dt
def clock(env, name, tick):
    while True:
        yield env.timeout(tick)

def run(env, sim_time):
    for name in ["clock%s"%i for i in range(10)]:
        env.process(clock(env, name, 0.5))
    stime = dt.now()
    env.run(until=sim_time)
    etime = dt.now()
    delta = etime - stime
    return delta.total_seconds()

In [13]:
env = simpy.Environment()
print("simpy: %f sec." % run(env, SIM_SIZE))

simpy: 15.875733 sec.


cdef しないと変わらんか。

In [15]:
env = simpyx.Environment()
print("simpyx: %f sec." % run(env, SIM_SIZE))

ValueError: <generator object at 0x7f46ab05a710> is not a generator.

なんかうごかないな。。。